In [2]:
import datetime as dt
import requests

import pandas as pd
import numpy as np
import os
import io
import re
import time
import datetime
from datetime import datetime
from bs4 import BeautifulSoup
import tabula 
import json
import pdfplumber


In [23]:
class Nodal:
    def __init__(self, df, access_token, blob_name):
        self.df = df
        self.access_token = access_token
        self.blob_name = blob_name

    def process_volume(self):
        nodal_volume_temp = self.df.copy()
        print(nodal_volume_temp.columns)
        nodal_volume_temp.insert(0, 'date', self.get_date())
        nodal_volume_temp['Market'] = 'Nodal OI and Volume'
        
        nodal_volume_temp.replace(' ', np.nan, inplace=True)
        self.common_processing(nodal_volume_temp)
        nodal_volume_temp['Open Interest'] =nodal_volume_temp['Open Interest'].astype(float,errors='ignore')
        nodal_volume_temp['OI Change'] =nodal_volume_temp['OI Change'].astype(float,errors='ignore')
        nodal_volume_temp['Total Volume'] =nodal_volume_temp['Total Volume'].astype(float,errors='ignore')
        nodal_volume_temp['Block Volume'] =nodal_volume_temp['Block Volume'].astype(float,errors='ignore')
        nodal_volume_temp.dropna(subset=['Open Interest', 'OI Change', 'Total Volume','Block Volume'], thresh=1, inplace=True)
        nodal_volume_temp.dropna(subset=['Code','Product'], thresh=1, inplace=True)
        #push_data2(commit_table['nodal_volume'], nodal_volume_temp, self.access_token)
        columns = ['date', 'Code', 'Product', 'Expiration', 'Open Interest', 'OI Change', 'Total Volume', 'Block Volume', 'Market']

# Drop columns that are not in the columns_to_keep list
        nodal_volume_temp = nodal_volume_temp[columns_to_keep]
        print(nodal_volume_temp.columns)

    def common_processing(self, nodal_temp):
        
        nodal_temp.dropna(subset=nodal_temp.columns[2:5], thresh=1, inplace=True)
        nodal_temp.replace(' ', np.nan, inplace=True)
        nodal_temp = nodal_temp.rename(columns={col: col.strip() for col in nodal_temp.columns})
       

    def get_date(self):
        if 'NEX_EOD' in self.blob_name:
            date_value = re.search(r"\d{4}(\d{2})(\d{2})", self.blob_name)
            if date_value:
                date_str = date_value.group(1) + " " + date_value.group(2) + " " + date_value.group(0)[:4]
                date = dt.datetime.strptime(date_str, "%m %d %Y").strftime("%d %b %Y")
                return date
            
        elif 'Nodal OI' in self.blob_name:
            date_value = re.search(r"(\d{1,2})\.(\d{1,2})\.(\d{4})", self.blob_name)
            if date_value:
                date_str = date_value.group(1) + " " + date_value.group(2) + " " + date_value.group(3)[:4]
                date = dt.datetime.strptime(date_str, "%m %d %Y").strftime("%d %b %Y")
                return date

In [24]:

df = pd.read_excel(r'D:\Nodal OI and Volume 11.16.2023.xlsx')
nodal = Nodal(df,'access_token', os.path.basename(r'D:\Nodal OI and Volume 11.16.2023.xlsx'))
if 'Volume' in os.path.basename(r'D:\Nodal OI and Volume 11.16.2023.xlsx'):
    nodal.process_volume()

Index(['Code', 'Product', 'Expiration', 'Open Interest', 'OI Change',
       'Total Volume', 'Block Volume', 'Unnamed: 7'],
      dtype='object')
Index(['date', 'Code', 'Product', 'Expiration', 'Open Interest', 'OI Change',
       'Total Volume', 'Block Volume', 'Market'],
      dtype='object')


In [10]:
file_name = os.path.basename(r'D:\Nodal OI and Volume 11.16.2023.xlsx')

# Now file_name contains the name of the file
print("File Name:", file_name)

File Name: Nodal OI and Volume 11.16.2023.xlsx
